In [1]:
import os 
import pandas as pd 
import numpy as np
from bold import boldAPI

# Download Sequences
if dna_sequences.csv exists then load it otherwise download species in `bold/bold_queries/species_names.txt`

In [5]:


dna_data_path = os.path.join('data', 'dna_sequences.csv')
if not os.path.exists(dna_data_path):
    species_names = list(set(boldAPI.species_from_species_namestxt())) # set to make sure its unique
    vfunc = np.vectorize(boldAPI.get_sequences)
    results = pd.concat(vfunc(species_names))
else: 
    results = pd.read_csv(dna_data_path)

results = results[results['sequence'].str.contains('N') == False] # if there is an N its an error
results['sequence'] = results['sequence'].apply(lambda x: x.replace('-', '')) # we dont care about the - its an alignment artifact and we can just remove them
results = results.drop_duplicates(subset='sequence') # there are a lot of duplicate sequences just drop all but the first

results.to_csv(dna_data_path, index=False)



downloading: Acropora millepora
You have exceeded your allowed request quota. If you wish to download large volume of data, please contact support@boldsystems.org for instruction on the process. 
downloading: Acropora millepora
You have exceeded your allowed request quota. If you wish to download large volume of data, please contact support@boldsystems.org for instruction on the process. 
downloading: pocillopora damicornis
You have exceeded your allowed request quota. If you wish to download large volume of data, please contact support@boldsystems.org for instruction on the process. 
downloading: favia fragum
You have exceeded your allowed request quota. If you wish to download large volume of data, please contact support@boldsystems.org for instruction on the process. 


OSError: Cannot save file into a non-existent directory: 'data'

In [21]:
import requests
process_id = 'GBFSE25693-24'
url = f'https://www.boldsystems.org/index.php/API_Public/specimen?ids={process_id}'


response = requests.get(url)

country_string = '</country'
filtered_list = [item for item in response.text.split('>')[1:] if country_string in item]
print(filtered_list[0].replace(country_string, ''))

Australia


In [44]:
from bold import boldAPI
import importlib
importlib.reload(boldAPI)


downloading: Acropora millepora


,species,BOLD_process_id,country,sequence
0,Acropora millepora,GBCI10386-19,Unknown,TACGTTATATTTAGTCTTTGGGATTGGGGCAGGCATGATTGGCACG...
0,Acropora millepora,GBFSE20474-24,Unrecoverable,TGCTTTTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE25693-24,Australia,GCTTTTACCCATTGAGGTGGGTGAGTTGGAGTGGTTGCGGCCTGCA...
0,Acropora millepora,GBFSE26948-24,Unrecoverable,TGCTTTTACCCAGTTACTTGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE28464-24,Unrecoverable,TGCTTCTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE31107-24,Unrecoverable,TGCTTCTACCCAATTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE35612-24,Unrecoverable,TGCTTCTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE43179-24,Unrecoverable,TGCTTTTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE45768-24,Unrecoverable,TGCTTCTACCCAGTTACACGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE47451-24,Unrecoverable,GTTTTTACCCGGTTATTCGGGTGAGTTGGAGTGGTCGCGGCCTGCC...


downloading: favia fragum


,species,BOLD_process_id,country,sequence
0,Favia fragum,GBCI0001-06,Panama,----------------------------------------------...
0,Favia fragum,GBCI0002-06,Brazil,----------------------------------------------...
0,Favia fragum,GBCI0234-06,Unknown,---------------------------------------ACTTTAT...
0,Favia fragum,GBCI0235-06,Unknown,---------------------------------------ACTTTAT...


downloading: pocillopora damicornis


In [36]:
boldAPI.get_sequences('acropora millepora')

acropora millepora


,species,BOLD_process_id,country,sequence


In [10]:
results

,Species,BOLDProcessId,Sequence
0,Abscondita terminalis,GBCL39854-19,CTTAATAAATGACTTTTTTCAACAAATCATAAAGACATTGGAACTT...
1,Abscondita terminalis,SPMIS002-22,AACTTTATATTTTATCTTCGGAATCTGAGCAGGAATATTAGGAACC...
2,Actias luna luna,GBGLS176-19,TTTTGATTTTTTGGGCATCCAGAAGTTTATATTTTAATTTTACCAG...
3,Actias luna,LGSM242-04,AACTTTATATTTTATTTTTGGAATTTGAGCAGGAATAGTAGGAACT...
4,Actias luna,LGSMC795-05,AACTTTATATTTTATTTTTGGAATTTGAGCAGGAATAGTAGGAACT...
...,...,...,...
41944,Zootermopsis nevadensis,GBMTG5022-16,ATGTTAATATTTTTTTATTTTTCTACCTTTTTTCTTTGTGGTTTGT...
41945,Zootermopsis nevadensis,GBMTG5022-16,ATGAATAAACCCATACGAAAAAACCACCCACTAATCAAAATTATAA...
41946,Zootermopsis nevadensis,GBMH10124-19,GAACAGAACTTGGACAACCAGGATCCTAATTGGAGACGACCAAATT...
41947,Zootermopsis nevadensis nuttingi,GBMH9919-19,GAACAGAACTTGGACAACCAGGATCCTAATTGGAGACGACCAAATT...


# get amino acid sequences for the DNA seqs

In [4]:
import os
import pandas as pd
from plmcluster import proteinextraction
# sequencePath = os.path.join('data', 'BOLD_sequences.csv')
# if os.path.exists(sequencePath) and:
#     results = pd.read_csv(sequencePath)

In [12]:
results.drop_duplicates(subset='Sequence', inplace=True) # there is a chance that after N and - being removed there are sequences that are duplicates now
results['AminoAcidSeq'] = results['Sequence'].apply(proteinextraction.get_aminoacid_seq)
results.to_csv(os.path.join('data', 'BOLD_seqs_with_protein.csv'), index=False)

/home/jnicolow/.conda/envs/numt/lib/python3.11/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
